## This Notebook will demonstrate different configurations and executions of LLM-Analyst

In [1]:
# Sytem level imports
import sys
import os
import json
sys.path.append('/Users/dan/Code/LLM/llm_analyst')

In [2]:
# Let's import llm_analyst content in one cell to make the rest of the code cleaner.
from llm_analyst.core.config import Config, DataSource
from llm_analyst.core.research_analyst import LLMAnalyst
from llm_analyst.core.research_editor import LLMEditor
from llm_analyst.core.research_publisher import LLMPublisher

### Demonstrate running LLM-Analyst on Local Data

In [4]:
## Let's run a simple research report against a set of local documemnts (Amy's Published Papers)
## Three things are required.
## 1. An active research topic 
## 2. A defined path to the local data to research against
## 3. We must indicate the data source (LocalStore, Web, SelectURLs)

# Requirement 1.
research_topic = "I would like to better understand how the metabolism of S-adenosylmethionine is linked to lipid metabolism and stress-responsive gene expression."

# Requirement 2.
config_params = {
    "internet_search":"ddg_search",
    "llm_provider"   :"openai",
    "llm_model"      :"gpt-4o-2024-05-13",
    "local_store_dir":"/Users/dan/Code/LLM/amy_papers"
}
config = Config()
config._set_values_for_config(config_params)



{'internet_search': <function llm_analyst.search_methods.internet_search.ddg_search(query, max_results=5)>,
 'embedding_provider': OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x132ddf1d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x10d5b1b90>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True),
 'llm_provider': llm_analyst.chat_models.openai.OPENA

In [5]:
# To execute some preliminary research we use the LLMAnalyst 
# request it conduct_research and then write_report
llm_analyst = LLMAnalyst(active_research_topic = research_topic, 
                         data_source = DataSource.LocalStore, 
                         config = config)

await llm_analyst.conduct_research()
research_state = await llm_analyst.write_report()


# Once the report is written we can ask the LLMPublisher to mak a pdf
llm_publisher = LLMPublisher(**research_state.dump(), config = config)
plublished_reseach_path = await llm_publisher.publish_to_pdf_file()

/Users/dan/miniforge3/envs/gpt-researcher/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Report written to /Users/dan/llm_analyst_out/Research-2024-06-03-0959030520.pdf


In [ ]:
## Let's build on the first research project and now build a "detailed report"
## A "detailed report" requires oversite therefore we will use an Editor v.s. an Analyst
## The key difference between an Editor and the Analyst is that
## the Editor will coordinate the efforts of multiple Analysts and 
## will utilize a specilized Report Writer to pull the final report together

## Inputs are the same as the Research Analyst Report above

## But we will explicitly define the publish directory.
## FYI this is the default so not actually required.
config_params["report_out_dir"] = "~/llm_analyst_out"

llm_editor = LLMEditor(active_research_topic = research_topic, 
                       data_source = DataSource.LocalStore,
                       config = config)

research_state = await llm_editor.create_detailed_report()

llm_publisher = LLMPublisher(**research_state.dump(), config = config)
plublished_reseach_path = await llm_publisher.publish_to_pdf_file()


In [ ]:
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time with milliseconds
formatted_date_time = now.strftime('%Y-%m-%d-%H%M%S%f')[:-2]

print(formatted_date_time)


In [ ]:
2024-06-03-0950413600
2024-06-03-0950561325